## 导入相关包

In [1]:
import pandas as pd 
import numpy as np
import math
import re
import jieba
from jieba.analyse import tfidf, textrank
from sklearn.decomposition import PCA

## 读取数据

In [35]:
# 文旅主题
themes = pd.read_table('data/extendtheme.txt', header=None)
# 所有实体
all_entity = pd.read_csv('data/所有实体.csv')
# OTA,UGC数据
df1 = pd.read_csv('data/游记攻略标注后的数据.csv')
df2 = pd.read_csv('data/酒店评论标注后的数据.csv')
df3 = pd.read_csv('data/景区标注后的数据.csv')
df4 = pd.read_csv('data/餐饮标注后的数据.csv')

In [37]:
# 评论保留到年
df1['发布时间'] = df1['发布时间'].apply(lambda t: str(t).split('-')[0])
df2['评论日期'] = df2['评论日期'].apply(lambda t: str(t).split('-')[0])
df3['评论日期'] = df3['评论日期'].apply(lambda t: str(t).split('-')[0])
df4['评论日期'] = df4['评论日期'].apply(lambda t: str(t).split('-')[0])

In [4]:
df4.head()

,餐饮评论ID,城市,餐饮名称,评论日期,评论内容,标题,实体编码
0,1001,茂名,盛香烧鹅（东方市场店）,2018,很好吃推荐！,主食3选1，有赠品,OOOOOO
1,1002,茂名,清香面包店（车田街店）,2018,超级好吃 老板又好 量又多,水果忌廉夹心蛋糕（二层）1个，约4磅，圆,OOOOOOOOOOOOOO
2,1003,茂名,功夫鸡排（光华北路店）,2018,好吃，不得不说比门口正对面那家好吃。服务态度也好。炸的皮松软，不会炸得很硬。,小吃6选1，提供免费WiFi,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
3,1004,茂名,茂名浪漫海岸温德姆酒店望海餐厅,2018,品种少，冷冻食品多。 7点就不再出食物，难以接受。,自助晚餐（浪漫海岸跨年音乐节）,OOOOOOOOOOOOOOOOOOOOOOOOO
4,1005,茂名,清香面包店（车田街店）,2018,味道不错，至少吃起来不腻，并且全家7个人吃还是挺多的了，就比较实惠咯,水果忌廉夹心蛋糕（二层）1个，约4磅，圆,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO


In [5]:
themes_list = themes.iloc[:, 0].tolist()
entity_list = all_entity['实体'].tolist()
entity_list.remove('B')
entity_list.remove('城市')

In [6]:
entity_list

['炒乌冬',
 '凤凰酒店',
 '兰州拉面',
 '甜品',
 '中国第一滩一家人公寓',
 '博贺港码头',
 '水东滨红树林景区',
 '油甘窝',
 '茂名站南广场',
 '徐闻港口码头酒店',
 '天马山生态旅游景区',
 'I',
 '蛇口码头',
 '白切鹅腿肉',
 '浪漫海岸旅游度假区',
 '博贺湾',
 '赤坎老街',
 '牛排',
 '碉楼',
 '生蚝',
 '油地公园',
 '菠萝包',
 '西湖公园',
 '贡园',
 '露天矿公园',
 '雷祖祠',
 '酱汁',
 '高钙大骨鸳鸯锅',
 '凉拌鹅颈肉',
 '放鸡岛海上游乐',
 'UNPOCO柏蔻',
 '晏镜岭',
 '红树林栈道',
 '薯条',
 '凉拌菜',
 '海滩',
 '玻璃桥',
 '蜂蜜芥末',
 '观鹭阁',
 '蛋挞',
 '北洛湾',
 '面包',
 '自助菜',
 '姻缘石',
 '凤梨',
 '滘尾角灯塔',
 '湖光岩',
 '马降龙',
 '沙街云吞',
 '火腿',
 '骑楼',
 '湛江大桥',
 '榕树',
 '立园',
 '情人桥',
 '十里军港',
 '炒牛河',
 '榴莲蛋糕',
 '炸鸡',
 '黄油',
 '荷花温泉',
 '雅米乐园',
 '架海灯塔',
 '露天矿生态公园',
 '牛肚',
 '泡芙',
 '奶油',
 '雷州古城',
 '脆皮热狗卷',
 '潘茂名纪念馆',
 '博物馆',
 '高州仙人洞景区',
 '海陵岛',
 '牙象大地艺术公园',
 '新湖公园',
 '关东煮',
 '煲仔饭',
 '温德姆酒店',
 '水东西湖',
 '油条碎',
 '小炒',
 '高凉郡古城',
 '玉湖',
 '百灵庙',
 '豆饼角',
 '东南码头',
 '水东湾',
 '牛肉',
 '文化广场',
 '桂花肠',
 '望海楼',
 '荔枝',
 '刺身',
 '自力村',
 '红树林',
 '甜点',
 '东湖',
 '水东港',
 '御水古温泉',
 '红树林湿地公园',
 '和牛盖饭',
 '榴莲千层',
 '三元塔',
 '寿司',
 '炖鹅骨头汤',
 '旺同竹海',
 '早茶',
 '温德姆至尊酒店',
 '灯楼角',
 '拉面',
 '窦州古城',
 '贵妃广场',
 '菠萝皮',
 'II',
 '茂名电白

## 旅游产品提取表

In [7]:
all_pros = []
pro_id = []
ids = []
ID = 1
all_id = {}
for i, v in zip(df2['酒店评论ID'], df2['评论内容'].astype(str)):
    for e in entity_list:
        if e in v:
            all_pros.append(e)
            pro_id.append('酒店评论-' + str(i))
            if e in all_id.keys():
                ids.append(all_id[e])
            else:
                ids.append('ID' + str(ID))
                all_id[e] = 'ID' + str(ID)
                ID += 1
for i, v in zip(df3['景区评论ID'], df3['评论内容'].astype(str)):
    for e in entity_list:
        if e in v:
            all_pros.append(e)
            pro_id.append('景区评论-' + str(i))
            if e in all_id.keys():
                ids.append(all_id[e])
            else:
                ids.append('ID' + str(ID))
                all_id[e] = 'ID' + str(ID)
                ID += 1
for i, v in zip(df4['餐饮评论ID'], df4['评论内容'].astype(str)):
    for e in entity_list:
        if e in v:
            all_pros.append(e)
            pro_id.append('餐饮评论-' + str(i))
            if e in all_id.keys():
                ids.append(all_id[e])
            else:
                ids.append('ID' + str(ID))
                all_id[e] = 'ID' + str(ID)
                ID += 1

In [8]:
len(pro_id), len(ids), len(all_pros)

(1805, 1805, 1805)

In [56]:
df_pros = pd.DataFrame({'语料ID':pro_id, '产品ID': ids, '产品名称': all_pros})
df_pros.head()

,语料ID,产品ID,产品名称
0,酒店评论-1025,ID1,海滩
1,酒店评论-1025,ID2,第一滩
2,酒店评论-1027,ID3,中心广场
3,酒店评论-1031,ID2,第一滩
4,酒店评论-1031,ID4,海鲜


In [57]:
df_pros

,语料ID,产品ID,产品名称
0,酒店评论-1025,ID1,海滩
1,酒店评论-1025,ID2,第一滩
2,酒店评论-1027,ID3,中心广场
3,酒店评论-1031,ID2,第一滩
4,酒店评论-1031,ID4,海鲜
...,...,...,...
1800,餐饮评论-7867,ID58,蔬菜
1801,餐饮评论-7886,ID61,米饭
1802,餐饮评论-7910,ID42,牛排
1803,餐饮评论-7910,ID16,饮料


In [58]:
df_pros.to_csv('data/result2-1.csv', index=False)

## 热度计算

In [38]:
texts = df1['正文'].astype(str).tolist() + df2['评论内容'].astype(str).tolist() + df3['评论内容'].astype(str).tolist() + df4['评论内容'].astype(str).tolist()
dates = df1['发布时间'].astype(str).tolist() + df2['评论日期'].astype(str).tolist() + df3['评论日期'].astype(str).tolist() + df4['评论日期'].astype(str).tolist()

In [40]:
# 所有语料
cropus = pd.DataFrame({'语料': texts, '年份': dates})
cropus.shape

(9574, 2)

In [41]:
# 添加自定义词典
for e in entity_list:
    jieba.add_word(e)

In [42]:
texts_cut = cropus['语料'].apply(jieba.lcut)

In [43]:
texts_cut

0       [1, 、, 茂名, 放鸡岛, 放鸡岛, （, 原名, 湾, 舟岛, ，, 又名, 汾洲岛,...
1       [【, 茂名, 在, 哪, 】, 我, 的, 大, 茂名, 位于, 广东, 西部, ，, 简...
2       [行程, 总览, 【, 地点, 】, 茂名, 【, 时间, 】, 3, 天, （, 实际, ...
3       [前言, 几分钟, 真的, 吃, 不遍, 茂名, ，, 题目, 只是, 一个, 噱头, 🔥,...
4       [茂名, 原本, 相对, 于, 广州, 而言, ，, 交通, 稍差, ，, 不, 适合, 周...
                              ...                        
9569      [吃, 得, 好, 饱, ，, 用餐, 感觉, 很, 好, ，, 下次, 还会, 继续, 来]
9570                        [回头客, 了, ，, 高州, 只, 喜欢, 来, 这里]
9571                                              [很, 好吃]
9572                                              [非常, 好]
9573                                     [好吃, 吃, 得, 好, 饱]
Name: 语料, Length: 9574, dtype: object

In [44]:
# 去用词
with open('data/stopword.txt', encoding='utf-8') as fp:
    words = fp.readlines()
    stopwords = [s.replace('\n', '').strip() for s in words] + ['\n', ' ', '茂名', '茂名市', '广东', '广东省', '中国',
                '国家', '全国',]
new_cut = texts_cut.apply(lambda text:[w for w in text if w not in stopwords])

In [45]:
cut_str = ''
for text in new_cut:
    cut_str += ' '.join(text)

### PMI互信息值

In [46]:
def pmi(texts=None, w=None, theme=None):
    '''
    texts：语料文档
    w：某个实体
    theme：主题词列表
    '''
    tl = len(texts)  # 文档长度
    el = len(theme)  # 主题长度
    pmi_value = 0
    for t in theme:
        w1 = 0
        w2 = 0
        w12 = 0
        for text in texts:
            if w in text:
                w1 += 1
            if t in text:
                w2 += 1
            if w in text and t in text:
                w12 += 1
        try:
            p = np.log2((w12/tl) / ((w1/tl) * (w2/tl)))
            if p == -np.inf:
                p = 0
            pmi_value += p # 计算PMI
        except:
            pass
    return round(pmi_value/el, 2)

In [54]:
entity_pmis = {}
years = []
for year in cropus['年份'].unique():
    for e in  set(df_pros['产品名称'].tolist()):
        p = pmi(cropus[cropus['年份'] == year]['语料'].values.tolist(), e, themes_list)
        entity_pmis[e+'-'+year] = p

D:\anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in log2


In [55]:
entity_pmis

{'叉烧饭-2018': 0.0,
 '奶油-2018': 0.02,
 '摩天轮-2018': 0.04,
 '和牛盖饭-2018': 0.0,
 '赤坎老街-2018': 0.0,
 '面包-2018': 0.6,
 '日式蟹肉卷-2018': 0.0,
 '湛江大桥-2018': 0.0,
 '海滨公园-2018': 0.09,
 '黄油-2018': 0.0,
 '露天矿生态公园-2018': 0.94,
 '刺身-2018': 0.0,
 '甜品-2018': 0.0,
 '海滩-2018': 0.18,
 '热狗-2018': 0.0,
 '野菠萝公园-2018': 0.0,
 '自助菜-2018': 0.0,
 '沙滩-2018': 0.31,
 '高州仙人洞景区-2018': 0.0,
 '菠萝皮-2018': 0.0,
 '粗薯-2018': 0.0,
 '温德姆酒店-2018': 0.0,
 '煲仔饭-2018': 0.0,
 '晏镜岭-2018': 0.0,
 '蔬菜-2018': 0.0,
 '红树林-2018': 0.82,
 '饮料-2018': 0.0,
 '海陵岛-2018': 0.25,
 '榴莲蛋糕-2018': 0.0,
 '米饭-2018': 0.0,
 '蛋挞-2018': 0.12,
 '博物馆-2018': 0.73,
 '荔枝-2018': 0.12,
 '薯条-2018': 0.0,
 '玻璃桥-2018': 0.0,
 '东湖-2018': 0.37,
 '玉湖-2018': 0.0,
 '拉面-2018': 0.0,
 '肉松-2018': 0.0,
 '御水古温泉-2018': 0.82,
 '炒牛河-2018': 0.0,
 '湖光岩-2018': 0.0,
 '高钙大骨鸳鸯锅-2018': 0.0,
 '水东湾-2018': 0.0,
 '骑楼-2018': 0.87,
 '炒乌冬-2018': 0.0,
 '十里军港-2018': 0.04,
 '无花果萨拉米披萨-2018': 0.0,
 '脆皮热狗卷-2018': 0.0,
 '关东煮-2018': 0.0,
 '文化广场-2018': 0.0,
 '小东江-2018': 0.93,
 '红树林栈道-2018': 0.0,
 '酱汁-2018': 0.

In [66]:
df_hat = pd.DataFrame({'产品名称': [i.split('-')[0] for i in entity_pmis.keys()],
                      'PMI': entity_pmis.values(),
                      '年份': [i.split('-')[1] for i in entity_pmis.keys()]})
df_hat['产品ID'] = list(df_hat['产品名称'].map(all_id))
df_hat.head()

,产品名称,PMI,年份,产品ID
0,叉烧饭,0.00,2018,ID56
1,奶油,0.02,2018,ID53
2,摩天轮,0.04,2018,ID24
3,和牛盖饭,0.00,2018,ID77
4,赤坎老街,0.00,2018,ID48


###  tf-idf值

In [21]:
tfidfs = tfidf(cut_str, topK=len(cut_str.split()), withWeight=True)
tfidf_dict = dict(tfidfs)

In [71]:
df_hat['TFIDF'] = list(df_hat['产品名称'].map(tfidf_dict))
df_hat['TFIDF'].fillna(0, inplace=True)

In [72]:
df_hat

,产品名称,PMI,年份,产品ID,TFIDF
0,叉烧饭,0.00,2018,ID56,0.001078
1,奶油,0.02,2018,ID53,0.009099
2,摩天轮,0.04,2018,ID24,0.002802
3,和牛盖饭,0.00,2018,ID77,0.000190
4,赤坎老街,0.00,2018,ID48,0.000254
...,...,...,...,...,...
355,第一滩,1.52,2021,ID2,0.015599
356,小炒,0.36,2021,ID50,0.000321
357,烧鸭饭,0.00,2021,ID57,0.001205
358,架海灯塔,0.40,2021,ID18,0.000127


## 主成分分析

In [93]:
pca = PCA(n_components=1)
pca_trans = pca.fit_transform(df_hat.loc[:, ['PMI', 'TFIDF']])
pca_list = np.array([[i[0] for i in pca_trans]])
pca_list 

array([[-0.34892246, -0.34892246, -0.34892221, -0.34891899,  0.4410947 ,
        -0.34892246,  0.21107672, -0.34892139, -0.34892172, -0.31889924,
        -0.20892258, -0.30891475, -0.32888203, -0.34892209,  1.1211084 ,
         0.55109224, -0.34891444,  0.3810882 ,  0.63107477,  0.47108733,
        -0.34891863, -0.34892227, -0.34891918, -0.34892169, -0.34892049,
        -0.19891993, -0.34892246,  1.24107387, -0.34891826,  0.52107778,
         1.1110807 , -0.18892312, -0.33889652, -0.34892227, -0.14892201,
        -0.30892189, -0.34892169, -0.34892246, -0.34889912, -0.22891377,
        -0.34892173, -0.34891115,  0.52107698, -0.34892173, -0.34891881,
        -0.34892246,  1.74109716,  0.12107615, -0.34891863,  1.08108823,
         0.88110939,  0.47108582, -0.34892227,  2.41107981, -0.0089228 ,
        -0.34889912,  0.16108331, -0.34891918, -0.34892227,  0.2410762 ,
         0.05107607, -0.09890034, -0.34892246, -0.30892226, -0.34891918,
         0.14107916,  0.16108036, -0.34891981, -0.3

In [94]:
def softmax(inMatrix):
    """
    softmax计算公式函数
    :param inMatrix: 矩阵数据
    :return:
    """
    m,n = np.shape(inMatrix)  #得到m,n(行，列)
    outMatrix = np.mat(np.zeros((m,n)))  #mat生成数组
    soft_sum = 0
    for idx in range(0,n):
        outMatrix[0,idx] = math.exp(inMatrix[0,idx])  #求幂运算，取e为底的指数计算变成非负
        soft_sum +=outMatrix[0,idx]   #求和运算
    for idx in range(0,n):
        outMatrix[0,idx] = outMatrix[0,idx] /soft_sum #然后除以所有项之后进行归一化
    return outMatrix

In [99]:
sol = softmax(pca_list).tolist()

In [100]:
hat = [round(i*33, 2) for i in sol[0]]

In [101]:
hat

[0.05,
 0.05,
 0.05,
 0.05,
 0.12,
 0.05,
 0.09,
 0.05,
 0.05,
 0.06,
 0.06,
 0.06,
 0.05,
 0.05,
 0.23,
 0.13,
 0.05,
 0.11,
 0.14,
 0.12,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.06,
 0.05,
 0.26,
 0.05,
 0.13,
 0.23,
 0.06,
 0.05,
 0.05,
 0.07,
 0.06,
 0.05,
 0.05,
 0.05,
 0.06,
 0.05,
 0.05,
 0.13,
 0.05,
 0.05,
 0.05,
 0.44,
 0.09,
 0.05,
 0.23,
 0.18,
 0.12,
 0.05,
 0.85,
 0.08,
 0.05,
 0.09,
 0.05,
 0.05,
 0.1,
 0.08,
 0.07,
 0.05,
 0.06,
 0.05,
 0.09,
 0.09,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.06,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.07,
 0.32,
 0.13,
 0.06,
 0.05,
 0.08,
 0.07,
 0.05,
 0.05,
 0.19,
 0.06,
 0.07,
 0.05,
 0.15,
 0.05,
 0.16,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.11,
 0.05,
 0.11,
 0.05,
 0.14,
 0.12,
 0.12,
 0.13,
 0.15,
 0.18,
 0.1,
 0.11,
 0.06,
 0.35,
 0.11,
 0.05,
 0.05,
 0.06,
 0.06,
 0.1,
 0.12,
 0.05,
 0.05,
 0.1,
 0.05,
 0.07,
 0.08,
 0.08,
 0.05,
 0.1,
 0.07,
 0.19,
 0.05,
 0.35,
 0.28,
 0.0

In [104]:
df_hat['产品热度'] = hat
df_hat.sort_values(by='产品热度', inplace=True, ascending=False)
df_hat

,产品名称,PMI,年份,产品ID,TFIDF,产品热度
321,小东江,0.15,2021,ID9,0.006041,0.85
26,饮料,0.00,2018,ID16,0.004543,0.44
281,刺身,0.00,2021,ID70,0.000747,0.41
33,薯条,0.00,2018,ID64,0.002920,0.38
50,文化广场,0.00,2018,ID6,0.001522,0.38
...,...,...,...,...,...,...
334,生蚝,1.00,2021,ID35,0.003361,0.05
175,第一滩,0.93,2019,ID2,0.015599,0.05
47,无花果萨拉米披萨,0.00,2018,ID81,0.000063,0.05
40,炒牛河,0.00,2018,ID73,0.000317,0.05


In [116]:
# 类型匹配
labels = {}
for pro in df_hat['产品名称'].unique():
    flag = True
    while flag:
        for text in df2['评论内容']:
            if pro in text:
                labels[pro] = '酒店'
                flag = False
                break
        for text in df2['酒店名称']:
            if pro in text:
                labels[pro] = '酒店'
                flag = False
                break
        for text in df3['评论内容']:
            if pro in text:
                labels[pro] = '景区'
                flag = False
                break
        for text in df4['评论内容']:
            if pro in text:
                labels[pro] = '餐饮'
                flag = False
                break    

In [119]:
df_hat['产品类型'] = list(df_hat['产品名称'].map(labels))
df_hat.head()

,产品名称,PMI,年份,产品ID,TFIDF,产品热度,产品类型
321,小东江,0.15,2021,ID9,0.006041,0.85,酒店
26,饮料,0.00,2018,ID16,0.004543,0.44,餐饮
281,刺身,0.00,2021,ID70,0.000747,0.41,餐饮
33,薯条,0.00,2018,ID64,0.002920,0.38,餐饮
50,文化广场,0.00,2018,ID6,0.001522,0.38,酒店


In [121]:
result2_2 = df_hat[['产品ID', '产品类型', '产品名称', '产品热度', '年份']]
result2_2.to_csv('data/result2-2.csv', index=False)